In [1]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
# print(str(Path().resolve().parent))
print(sys.path)
import os
os.environ['OMP_NUM_THREADS'] = "1"

['C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311', 'd:\\github\\scikit-quri\\pyenv', '', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\win32', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\win32\\lib', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\Pythonwin', 'D:\\github\\scikit-quri']


In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent.parent))

# import better_exceptions
import numpy as np
from typing import Optional
from quri_parts.circuit.utils.circuit_drawer import draw_circuit
from quri_parts.circuit import UnboundParametricQuantumCircuit,LinearMappedUnboundParametricQuantumCircuit
from numpy.random import default_rng
from scikit_quri.circuit.circuit import LearningCircuit

from qulacs import Observable
from quri_parts.core.operator import Operator, pauli_label
from quri_parts.core.estimator.gradient import (
    create_parameter_shift_gradient_estimator,
    create_numerical_gradient_estimator,)
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_estimator,
    create_qulacs_vector_parametric_estimator,
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.algo.optimizer import Adam, LBFGS




In [3]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
x = df.loc[:, ["petal length (cm)", "petal width (cm)"]]
x_train,x_test,y_train,y_test = train_test_split(x,iris.target,test_size=0.25,random_state=0)

x_train = x_train.to_numpy()

from quri_parts.circuit.utils.circuit_drawer import draw_circuit
import matplotlib.pyplot as plt
n_qubits = 5
num_class = 3
print(x_train)

[[4.2 1.5]
 [4.  1.2]
 [5.5 2.1]
 [1.3 0.2]
 [5.1 2.3]
 [1.6 0.6]
 [1.5 0.2]
 [3.5 1. ]
 [5.5 1.8]
 [5.7 2.5]
 [5.  1.5]
 [5.8 1.8]
 [3.9 1.1]
 [6.1 2.3]
 [4.7 1.6]
 [3.8 1.1]
 [4.9 1.8]
 [5.1 1.5]
 [4.5 1.7]
 [5.  1.9]
 [4.7 1.4]
 [5.2 2. ]
 [4.5 1.6]
 [1.6 0.2]
 [5.1 1.9]
 [4.2 1.3]
 [3.6 1.3]
 [4.  1.3]
 [4.6 1.4]
 [6.  1.8]
 [1.5 0.2]
 [1.1 0.1]
 [5.3 1.9]
 [4.2 1.2]
 [1.7 0.2]
 [1.5 0.4]
 [4.9 1.5]
 [1.5 0.2]
 [5.1 1.8]
 [3.  1.1]
 [1.4 0.3]
 [4.5 1.5]
 [6.1 2.5]
 [4.2 1.3]
 [1.4 0.1]
 [5.9 2.1]
 [5.7 2.3]
 [5.8 2.2]
 [5.6 2.1]
 [1.6 0.2]
 [1.6 0.2]
 [5.1 2. ]
 [5.7 2.1]
 [1.3 0.3]
 [5.4 2.3]
 [1.4 0.2]
 [5.  2. ]
 [5.4 2.1]
 [1.3 0.2]
 [1.4 0.2]
 [5.8 1.6]
 [1.4 0.3]
 [1.3 0.2]
 [1.7 0.4]
 [4.  1.3]
 [5.9 2.3]
 [6.6 2.1]
 [1.4 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [4.5 1.3]
 [4.4 1.4]
 [1.2 0.2]
 [1.7 0.5]
 [4.3 1.3]
 [1.5 0.4]
 [6.9 2.3]
 [3.3 1. ]
 [6.4 2. ]
 [4.4 1.4]
 [1.5 0.1]
 [4.8 1.8]
 [1.2 0.2]
 [6.7 2. ]
 [1.5 0.3]
 [1.6 0.2]
 [6.1 1.9]
 [1.4 0.2]
 [5.6 2.4]
 [4.1 1.3]
 [3.9 1.2]

In [4]:
from scikit_quri.circuit.pre_defined import create_qcl_ansatz,create_farhi_neven_ansatz
# circuit = create_farhi_neven_ansatz(n_qubits, 2,0)
circuit = create_qcl_ansatz(n_qubits,3,1.)
draw_circuit(circuit.circuit)
print(circuit.circuit.parameter_count)

   ___     ___     ___     ___     ___     ___     ___     ___     ___     ___  
  |PRY|   |PRZ|   |Mat|   |PRX|   |PRZ|   |PRX|   |Mat|   |PRX|   |PRZ|   |PRX| 
--|0  |---|1  |---|10 |---|11 |---|12 |---|13 |---|26 |---|27 |---|28 |---|29 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    ___     ___     ___    |   |    ___     ___     ___  
  |PRY|   |PRZ|   |   |   |PRX|   |PRZ|   |PRX|   |   |   |PRX|   |PRZ|   |PRX| 
--|2  |---|3  |---|   |---|14 |---|15 |---|16 |---|   |---|30 |---|31 |---|32 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    ___     ___     ___    |   |    ___     ___     ___  
  |PRY|   |PRZ|   |   |   |PRX|   |PRZ|   |PRX|   |   |   |PRX|   |PRZ|   |PRX| 
--|4  |---|5  |---|   |---|17 |---|18 |---|19 |---|   |---|33 |---|34 |---|35 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    _

In [5]:
print(f"{x_train[3]=}")
# init_params = [-0.6459906318187236, 3.5726052125794032, -3.7261666316598814, -6.381331778040319, 4.448677843102995, 3.4708870076700777, -5.070968615387754, -6.328852747092929]
# init_params =[4.002148315014479, 1.6951199159934145, 0.25744424357926954, 0.10384619671527331, 5.109927617709579, 5.735012432197602, 3.811604993109835, 4.583562073612696, 3.415696558991173, 5.8752333142921085, 5.126159064066656, 0.017206504032783308, 5.3872299529679575, 0.21102439329246717, 4.584560380312186]
init_params = [np.random.rand() for _ in range(circuit.n_learning_params)]
unbound_circuit = circuit.bind_input_and_parameters(x_train[0],init_params)
for gate in unbound_circuit.gates:
    print(gate.name,gate.params)
    if gate.name == "UnitaryMatrix":
        # print(gate.unitary_matrix)
        pass

x_train[3]=array([1.3, 0.2])
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
UnitaryMatrix ()
RX (0.5176484364446301,)
RZ (0.5360055810611856,)
RX (0.9487184762335936,)
RX (0.5536941626393868,)
RZ (0.06285371071116486,)
RX (0.49543575290152086,)
RX (0.8301331233668011,)
RZ (0.375498059350147,)
RX (0.9123391342755798,)
RX (0.5996177748800194,)
RZ (0.5256829014296289,)
RX (0.7656943125798237,)
RX (0.6542139425991226,)
RZ (0.7738134489941001,)
RX (0.045747321243855055,)
UnitaryMatrix ()
RX (0.40719248109594974,)
RZ (0.9530732150977393,)
RX (0.2724975579248442,)
RX (0.08179782576057881,)
RZ (0.9786129756196896,)
RX (0.01663549177071033,)
RX (0.8289160797388516,)
RZ (0.23433810177271241,)
RX (0.6539253863155442,)
RX (0.36113585491744293,)
RZ (0.4646458061506541,)
RX (0.22759208091850747,)
RX (0.11002810815348807,)
RZ (0.6339505077459019,)
RX (0.8300713839455703,)
Un

In [6]:
from quri_parts.core.state import quantum_state
param_circuit_state = quantum_state(n_qubits=5,circuit=circuit.circuit)
params = circuit.generate_bound_params(x_train[0],init_params)
circuit_state = param_circuit_state.bind_parameters(params)

from quri_parts.qulacs.estimator import create_qulacs_vector_estimator
from quri_parts.core.operator import Operator, pauli_label

op = Operator({pauli_label("Z0"): 1.0})
# estimator = create_qulacs_vector_estimator()
estimator = create_qulacs_vector_concurrent_estimator()
estimate = create_qulacs_vector_estimator()(op, circuit_state)
print(f"{estimate.value.real*2.2=}")

estimate.value.real*2.2=-0.19044212496884444


In [7]:
from scikit_quri.qnn.classifier import QNNClassifier

ops = []
for i in range(num_class):
    op = Operator({pauli_label(f"Z {i}"):1.0})
    ops.append(op)

adam = Adam()
estimator = create_qulacs_vector_concurrent_estimator()
gradient_estimator = create_numerical_gradient_estimator(create_qulacs_vector_concurrent_parametric_estimator(),delta=1e-10)
qnn = QNNClassifier(circuit,num_class,estimator,gradient_estimator,adam,ops,)

In [8]:
res = qnn._predict_inner(x_train,params=init_params)
print(res)

[[-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.81135473 -0.31265864  0.85786669]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.73838231  0.01246474  0.61844976]
 [-0.81135473 -0.31265864  0.85786669]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.81135473 -0.31265864  0.85786669]
 [-0.19044212  0.14011661 -0.00310286]
 [-0.19044212  0.14011661

In [9]:
res = qnn.cost_func_grad(x_train,y_train,init_params)


In [10]:
qnn.fit(x_train,y_train,maxiter=50)

init_params=array([0.46515283, 3.59096513, 3.42143429, 0.77373952, 2.2419391 ,
       1.58077867, 4.69624959, 6.18469545, 5.72921076, 3.87282381,
       1.77142652, 1.73594127, 4.63241693, 5.22830499, 4.99535162,
       4.69330783, 4.82350662, 5.66976434, 0.41845587, 5.68465081,
       3.60673665, 1.91937354, 0.18149148, 5.40686298, 3.81056389,
       5.87482245, 5.43029683, 5.33893091, 2.44792184, 2.16457059,
       1.37047898, 3.55743391, 0.41243228, 1.3108638 , 0.39741038,
       4.59605013, 3.48766238, 3.95680667, 3.58748959, 0.32255409,
       4.06395745, 4.13952152, 4.47161283, 1.43568012, 0.44421621])
cost_func(init_params)=1.3029833989071327
self._estimate_grad(x_scaled,init_params)=array([[[-5.47389911e-02, -4.78808659e-02, -1.02526321e-02, ...,
          2.77555756e-07, -4.16333634e-07, -2.77555756e-07],
        [-2.01124395e-01, -6.36682373e-02,  1.48401846e-01, ...,
         -8.32667268e-07, -2.49800181e-06,  1.38777878e-06],
        [-3.13382653e-02, -2.74471557e-01,  7.63

In [11]:
y_pred = qnn.predict(x_test)
from scipy.special import softmax
from sklearn.metrics import log_loss
print(y_pred)
print(y_test)

y_pred_sm = softmax(y_pred,axis=1)
print(y_pred_sm)
log = log_loss(y_test,y_pred_sm)
# log = log_loss(y_test,y_pred)
print(log)

[[-0.887323   -0.03240685  1.08574469]
 [-0.72460462  1.20299792 -0.3261124 ]
 [ 1.50515706 -0.36929469 -1.40298467]
 [-1.20861694 -0.23380591  0.89980388]
 [ 1.46283313 -0.30426302 -1.38882583]
 [-0.14894881  0.09165763  0.59224593]
 [ 1.45459296 -0.32904432 -1.44418614]
 [-1.14726105  0.92786404  0.3793848 ]
 [-1.1094342   0.9480597   0.33808432]
 [-0.92668425  1.20124016 -0.09529282]
 [-1.16458545  0.58367346  0.64780383]
 [-1.11790144  0.99158555  0.29247594]
 [-0.97088291  1.05064897  0.13032976]
 [-1.13329388  0.96058848  0.335995  ]
 [-1.09748926  0.98186214  0.29379477]
 [ 1.2714791  -0.45578321 -1.00137922]
 [-1.11790144  0.99158555  0.29247594]
 [-0.93466119  1.13254222 -0.00479878]
 [ 1.47918004 -0.2379939  -1.48842682]
 [ 1.38995361 -0.20399191 -1.46643692]
 [-1.27787797  0.37229504  0.92449757]
 [-1.11790144  0.99158555  0.29247594]
 [ 1.06063888 -0.14870357 -1.14696431]
 [ 1.50515706 -0.36929469 -1.40298467]
 [-1.25774104  0.64317279  0.69843974]
 [ 0.68972039 -0.6204055 

d:\github\scikit-quri\pyenv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
